# Bírósági Határozatok - Exploratív Adatanalízis (EDA)

Ez a notebook a jelenlegi pipeline-ban található processed_docs.jsonl metadatokat elemzi.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from pathlib import Path

# Projekt konfiguráció betöltése
import sys
project_root = Path(__file__).parent.parent
sys.path.insert(0, str(project_root))
try:
    from configs import config
except ImportError:
    print("Figyelem: configs modul nem található. Használja a default értékeket.")
    class Config:
        PROCESSED_DOCS_LIST = project_root / "data" / "processed" / "processed_docs.jsonl"
    config = Config()

# Matplotlib beállítások
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 12

# Adatok betöltése a jelenlegi pipeline-ból
print("Adatok betöltése a processed_docs.jsonl-ból...")
df = None
try:
    processed_docs_file = getattr(config, 'PROCESSED_DOCS_LIST', None)
    if processed_docs_file and Path(processed_docs_file).exists():
        processed_docs_list = []
        with open(processed_docs_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    doc = json.loads(line.strip())
                    processed_docs_list.append(doc)
                except json.JSONDecodeError:
                    continue

        if processed_docs_list:
            df = pd.DataFrame(processed_docs_list)
            print(f"{len(df)} dokumentum betöltve sikeresen.")
        else:
            print("Figyelem: Nem találhatóak feldolgozható dokumentumok.")
    else:
        print("Figyelem: processed_docs.jsonl fájl nem található.")

except Exception as e:
    print(f"Hiba történt a processed_docs.jsonl betöltése során: {e}")
    print("Használja a default értékeket vagy futtassa a build pipeline-t.")

NameError: name '__file__' is not defined

## 1. Adatok Betöltése

In [ ]:
# Az adatkeret előkészítése: alapvető információk
if df is not None and not df.empty:
    print(f"Eredeti oszlopok: {df.columns.tolist()}")

    # A processed_docs.jsonl metadatok elemzése
    print(f"\nMegmaradt oszlopok: {df.columns.tolist()}")
    print("\nA processed_docs.jsonl csak metadatokat tartalmaz (doc_id, raw_path, chunk_count, timestamp).")
    print("A teljes szövegek a chunks.jsonl-ben vannak.")

    # Oszlopok átnevezése ha szükséges
    if 'raw_path' in df.columns:
        print("Raw path oszlop található - ez a dokumentumok eredeti elérési útja.")

    if 'chunk_count' in df.columns:
        try:
            print(f"Chunk-ok száma statisztikák: {df['chunk_count'].describe()}")
        except Exception as e:
            print(f"Hiba a chunk_count statisztikák kiszámítása során: {e}")

    print(f"\nAdatok típusa: {type(df)}")
    print(f"Adatok alakja: {df.shape}")

else:
    print("Figyelem: 'df' adatkeret nem létezik vagy üres. Az adatok betöltése valószínűleg sikertelen volt.")
    print("Futtassa a build pipeline-t vagy ellenőrizze a fájlokat.")

## 2. Alapvető Információk és Statisztikák

In [ ]:
if 'df' in locals():
    print("Adatkeret mérete (sorok, oszlopok):", df.shape)
    print("\nElső 5 sor:")
    display(df.head())
    print("\nAdattípusok és nem-null értékek:")
    df.info()
    print("\nLeíró statisztikák (numerikus oszlopok):")
    display(df.describe())
    print("\nLeíró statisztikák (kategorikus és egyéb oszlopok):")
    display(df.describe(include=['object', 'boolean']))

## 3. Hiányzó Értékek Elemzése

In [ ]:
if 'df' in locals():
    missing_values = df.isnull().sum()
    missing_percent = (missing_values / len(df)) * 100
    missing_df = pd.DataFrame({'Darabszám': missing_values, 'Százalék': missing_percent})
    missing_df = missing_df[missing_df['Darabszám'] > 0].sort_values(by='Százalék', ascending=False)

    print("Hiányzó értékek oszloponként:")
    display(missing_df)

    # Hiányzó értékek vizualizációja (heatmap)
    plt.figure(figsize=(15, 8))
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
    plt.title('Hiányzó értékek eloszlása az adatkeretben')
    plt.show()

## 4. Szövegelemzés

In [ ]:
if 'df' in locals() and 'text' in df.columns:
    # Szöveghossz számítása (karakterekben)
    df['text_length'] = df['text'].fillna('').astype(str).apply(len)

    print("Szöveghossz statisztikák (karakterek):")
    display(df['text_length'].describe())

    # Kiugró értékek szűrése a jobb vizualizáció érdekében
    # Például a 99. percentilis alatti értékek megjelenítése
    quantile_99 = df['text_length'].quantile(0.99)
    df_filtered_length = df[df['text_length'] < quantile_99]
    print(f"\nAz adatok 99%-a {quantile_99:.0f} karakternél rövidebb szöveget tartalmaz.")
    print(f"A hisztogram csak ezeket az adatokat mutatja ( {len(df_filtered_length)} db ).")

    # Szöveghossz eloszlás vizualizáció (szűrt adatokon)
    plt.figure(figsize=(12, 6))
    sns.histplot(data=df_filtered_length, x='text_length', bins=100, kde=False)
    plt.title(f'Szöveghossz eloszlása (karakterekben) - {quantile_99:.0f} karakternél rövidebb szövegek')
    plt.xlabel('Szöveghossz (karakter)')
    plt.ylabel('Dokumentumok száma')
    plt.grid(True)
    plt.show()

    # Rövid szövegek vizsgálata
    short_text_threshold = 50
    short_texts = df[df['text_length'] < short_text_threshold]
    print(f"\n{len(short_texts)} db {short_text_threshold} karakternél rövidebb dokumentum található.")
    if not short_texts.empty:
        print(f"Példák rövid szövegekre (max 10):")
        display(short_texts[['doc_id', 'text', 'text_length']].head(10))

    # Üres szövegek vizsgálata
    empty_texts = df[df['text_length'] == 0]
    print(f"\n{len(empty_texts)} db üres szövegű dokumentum található.")
    if not empty_texts.empty:
        print(f"Példák üres szövegekre (max 10):")
        display(empty_texts[['doc_id', 'text_length']].head(10))

## 5. Kategorikus Változók Elemzése

In [ ]:
def plot_top_categories(df, column_name, top_n=20):
    """Segédfüggvény a leggyakoribb kategóriák megjelenítésére."""
    if column_name not in df.columns:
        print(f"Hiba: '{column_name}' oszlop nem található.")
        return

    counts = df[column_name].value_counts()
    print(f"\n'{column_name}' egyedi értékeinek száma: {counts.nunique()}")
    print(f"Leggyakoribb {top_n} érték:")
    display(counts.head(top_n))

    plt.figure(figsize=(12, 6))
    counts.head(top_n).plot(kind='bar')
    plt.title(f'Dokumentumok megoszlása - {column_name} (Top {top_n})')
    plt.xlabel(column_name)
    plt.ylabel('Dokumentumok száma')
    plt.xticks(rotation=75, ha='right')
    plt.grid(axis='y')
    plt.tight_layout() # Ensure labels fit
    plt.show()

if 'df' in locals():
    # Bíróság elemzése
    plot_top_categories(df, 'birosag', top_n=30)

    # Jogterület elemzése
    # Megjegyzés: A JogTerulet lehet komplex (pl. lista). A preprocess script stringgé alakítja.
    # Ha mégsem, akkor itt további tisztításra lehet szükség.
    plot_top_categories(df, 'JogTerulet', top_n=20)

    # HatarozatEve elemzése
    if 'HatarozatEve' in df.columns:
        # Érvényes évszámok szűrése (pl. 2005-től napjainkig) és integer típussá alakítás
        # Use Int64 to handle potential NaNs while keeping integer type
        df['HatarozatEve_num'] = pd.to_numeric(df['HatarozatEve'], errors='coerce').astype('Int64')
        valid_years_mask = (df['HatarozatEve_num'] >= 2005) & (df['HatarozatEve_num'] <= pd.Timestamp.now().year)
        valid_years = df.loc[valid_years_mask, 'HatarozatEve_num']

        print(f"\nÉrvényes 'HatarozatEve' értékek száma (2005 óta): {len(valid_years)}")
        print(f"Érvénytelen vagy hiányzó 'HatarozatEve' értékek száma: {len(df) - len(valid_years)}")

        if not valid_years.empty:
            plt.figure(figsize=(12, 6))
            # Use discrete=True for integer years to center bars on ticks
            min_year = valid_years.min()
            max_year = valid_years.max()
            # Let histplot handle bins for discrete data
            sns.histplot(x=valid_years, kde=False, discrete=True)
            plt.title('Dokumentumok megoszlása HatarozatEve szerint (2005 óta)')
            plt.xlabel('Határozat Éve')
            plt.ylabel('Dokumentumok száma')
            # Ensure ticks are placed at integer years
            tick_step = max(1, (max_year - min_year) // 20) # Adjust step for readability
            plt.xticks(ticks=range(min_year, max_year + 1, tick_step), rotation=0) # Set rotation to 0 for clarity if possible
            plt.xlim(min_year - 0.5, max_year + 0.5) # Keep slight padding
            plt.grid(True)
            plt.show()
        else:
            print("\n'HatarozatEve' oszlop nem található vagy nincsenek érvényes évszámok 2005 óta.")

## 5.1. Bíróság és Jogterület Kapcsolata

In [ ]:
if 'df' in locals() and 'birosag' in df.columns and 'JogTerulet' in df.columns:
    print("\nBíróság és Jogterület kapcsolata:")
    top_n_birosag = 20
    top_m_jogterulet = 15
    top_birosagok = df['birosag'].value_counts().nlargest(top_n_birosag).index
    top_jogteruletek = df['JogTerulet'].value_counts().nlargest(top_m_jogterulet).index
    df_filtered = df[df['birosag'].isin(top_birosagok) & df['JogTerulet'].isin(top_jogteruletek)]
    if not df_filtered.empty:
        crosstab_bj = pd.crosstab(df_filtered['birosag'], df_filtered['JogTerulet'])
        plt.figure(figsize=(15, 10))
        sns.heatmap(crosstab_bj, cmap="viridis", annot=False, fmt="d")
        plt.title(f'Bíróságok és Jogterületek kapcsolata (Top {top_n_birosag} bíróság, Top {top_m_jogterulet} jogterület)')
        plt.xlabel('Jogterület')
        plt.ylabel('Bíróság')
        plt.xticks(rotation=60, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()
    else:
        print("Nem található elég adat a szűrt bíróságokhoz és jogterületekhez a kereszttábla elkészítéséhez.")

## 5.2. Szöveghossz és Kategóriák

In [ ]:
if 'df' in locals() and 'text_length' in df.columns:
    top_n = 15
    if 'birosag' in df.columns:
        top_birosagok = df['birosag'].value_counts().nlargest(top_n).index
        df_filtered_birosag = df[df['birosag'].isin(top_birosagok)]
        quantile_99 = df_filtered_birosag['text_length'].quantile(0.99)
        df_plot_birosag = df_filtered_birosag[df_filtered_birosag['text_length'] < quantile_99]
        plt.figure(figsize=(15, 8))
        sns.boxplot(data=df_plot_birosag, x='birosag', y='text_length', order=top_birosagok)
        plt.title(f'Szöveghossz eloszlása a leggyakoribb {top_n} bíróság szerint (99% alatti értékek)')
        plt.xlabel('Bíróság')
        plt.ylabel('Szöveghossz (karakter)')
        plt.xticks(rotation=75, ha='right')
        plt.tight_layout()
        plt.show()
    if 'JogTerulet' in df.columns:
        top_jogteruletek = df['JogTerulet'].value_counts().nlargest(top_n).index
        df_filtered_jogterulet = df[df['JogTerulet'].isin(top_jogteruletek)]
        quantile_99_jt = df_filtered_jogterulet['text_length'].quantile(0.99)
        df_plot_jogterulet = df_filtered_jogterulet[df_filtered_jogterulet['text_length'] < quantile_99_jt]
        plt.figure(figsize=(15, 8))
        sns.boxplot(data=df_plot_jogterulet, x='JogTerulet', y='text_length', order=top_jogteruletek)
        plt.title(f'Szöveghossz eloszlása a leggyakoribb {top_n} jogterület szerint (99% alatti értékek)')
        plt.xlabel('Jogterület')
        plt.ylabel('Szöveghossz (karakter)')
        plt.xticks(rotation=75, ha='right')
        plt.tight_layout()
        plt.show()

## 5.3. Időbeli Trendek

In [ ]:
if 'df' in locals() and 'HatarozatEve_num' in df.columns and 'JogTerulet' in df.columns:
    print("\nJogterületek gyakoriságának változása az évek során:")
    # Ensure HatarozatEve_num is Int64 before filtering
    if not pd.api.types.is_integer_dtype(df['HatarozatEve_num']):
         df['HatarozatEve_num'] = pd.to_numeric(df['HatarozatEve'], errors='coerce').astype('Int64')

    min_analysis_year = 2005
    current_year = pd.Timestamp.now().year
    valid_years_mask = (df['HatarozatEve_num'] >= min_analysis_year) & (df['HatarozatEve_num'] <= current_year)
    top_m_jogterulet = 5
    top_jogteruletek = df['JogTerulet'].value_counts().nlargest(top_m_jogterulet).index
    df_filtered_time = df[valid_years_mask & df['JogTerulet'].isin(top_jogteruletek)].copy()

    if not df_filtered_time.empty:
        trends = df_filtered_time.groupby(['HatarozatEve_num', 'JogTerulet']).size().unstack(fill_value=0)
        plt.figure(figsize=(15, 8))
        trends.plot(kind='line', marker='o', ax=plt.gca()) # Added marker for clarity
        plt.title(f'Leggyakoribb {top_m_jogterulet} jogterülethez tartozó dokumentumok számának alakulása évente ({min_analysis_year}-{current_year})')
        plt.xlabel('Határozathozatal Éve')
        plt.ylabel('Dokumentumok száma')
        plt.xticks(range(min_analysis_year, current_year + 1, max(1, (current_year - min_analysis_year) // 15))) # Adjust tick frequency
        plt.xlim(min_analysis_year - 0.5, current_year + 0.5) # Adjust xlim slightly
        plt.legend(title='Jogterület')
        plt.grid(True)
        plt.tight_layout()
        plt.show()
    else:
        print(f"Nem található elég adat a top {top_m_jogterulet} jogterület időbeli trendjének vizsgálatához {min_analysis_year} óta.")

## 6. Azonosítók Elemzése

In [ ]:
if 'df' in locals():
    for id_col in ['doc_id', 'Azonosito', 'EgyediAzonosito']:
        if id_col in df.columns:
            print(f"\n'{id_col}' elemzése:")
            uniqueness = df[id_col].nunique()
            total_rows = len(df)
            duplicates = total_rows - uniqueness
            print(f"  Összes sor: {total_rows}")
            print(f"  Egyedi értékek: {uniqueness}")
            print(f"  Duplikált értékek (vagy hiányzó): {duplicates}")
            if duplicates > 0:
                duplicated_ids = df[df.duplicated(subset=[id_col], keep=False)][id_col].value_counts()
                print(f"  Leggyakoribb duplikált/hiányzó '{id_col}' értékek:")
                display(duplicated_ids[duplicated_ids > 1].head())
        else:
            print(f"\n'{id_col}' oszlop nem található.")

## 7. Kapcsolódó Határozatok Elemzése

In [ ]:
if 'df' in locals():
    # KapcsolodoHatarozatok (JSON stringként tárolva)
    if 'KapcsolodoHatarozatok' in df.columns:
        print("\n'KapcsolodoHatarozatok' elemzése:")
        # Számoljuk, hány rekordnak van nem-üres/nem-null kapcs. hat. listája
        # Feltételezzük, hogy a preprocess script NaN-t, '[]'-t vagy más stringet/listát menthet
        # Ez a sor ellenőrzi, hogy az érték nem NaN ÉS nem az üres lista string reprezentációja
        has_related = df['KapcsolodoHatarozatok'].notna() & (df['KapcsolodoHatarozatok'] != '[]')
        print(f"  Dokumentumok száma kapcsolódó határozattal (nem üres lista/nem NaN): {has_related.sum()}")
        print(f"  Dokumentumok száma kapcsolódó határozat nélkül (NaN vagy '[]'): {len(df) - has_related.sum()}")
    else:
        print("\n'KapcsolodoHatarozatok' oszlop nem található.")

    # Első kapcsolódó bíróság elemzése
    if 'Kapcsolódó Bíróság' in df.columns:
        plot_top_categories(df, 'Kapcsolódó Bíróság', top_n=20)
    else:
        print("\n'Kapcsolódó Bíróság' oszlop nem található.")

    # Első kapcsolódó ügyszám elemzése
    if 'Kapcsolódó Ügyszám' in df.columns:
        print("\n'Kapcsolódó Ügyszám' elemzése:")
        # Először csak a jelenlétét nézzük
        has_all_ugyszam = df['Kapcsolódó Ügyszám'].notna().sum()
        print(f"  Dokumentumok száma első kapcsolódó ügyszámmal: {has_all_ugyszam}")
        print(f"  Dokumentumok száma első kapcsolódó ügyszám nélkül: {len(df) - has_all_ugyszam}")
        # Most vizualizáljuk a leggyakoribb ügyszámokat
        plot_top_categories(df, 'Kapcsolódó Ügyszám', top_n=20)
    else:
        print("\n'Kapcsolódó Ügyszám' oszlop nem található.")

## 7.1. Jogszabályhelyek Mélyebb Elemzése

In [ ]:
import ast
if 'df' in locals() and 'Jogszabalyhelyek' in df.columns:
    print("\n'Jogszabalyhelyek' elemzése:")

    # --- DIAGNOSZTICA START ---
    print("\nDiagnosztika: Első 10 nem-üres/nem-'[]' Jogszabalyhelyek érték:")
    sample_values = df[df['Jogszabalyhelyek'].notna() & (df['Jogszabalyhelyek'] != '[]')]['Jogszabalyhelyek'].head(10)
    if not sample_values.empty:
        for i, val in enumerate(sample_values):
            print(f"  {i+1}: Type={type(val)}, Value='{val}'")
    else:
        print("  Nem található minta a diagnosztikához (minden érték NaN vagy '[]').")
    print("--- DIAGNOSZTICA END ---\n")
    # --- END DIAGNOSTICS ---

    def parse_jogszabaly(value):
        if pd.isna(value) or not isinstance(value, str) or value.strip() == '[]' or not value.strip():
            return []
        try:
            # Darabolás a '</br>' mentén, majd felesleges szóközök eltávolítása
            items = value.split('</br>')
            # Üres elemek kiszűrése és szóközök eltávolítása az elemek elejéről/végéről
            cleaned_items = [item.strip() for item in items if item.strip()]
            return cleaned_items
        except Exception as e:
            # Váratlan hiba esetén is üres listát adunk vissza
            print(f"Hiba a jogszabályhely feldolgozása közben: {e}, Érték: {value}")
            return []

    df['parsed_jogszabalyhelyek'] = df['Jogszabalyhelyek'].apply(parse_jogszabaly)
    jogszabaly_series = df['parsed_jogszabalyhelyek'].explode().dropna()
    print(f"\nÖsszesen {len(jogszabaly_series)} jogszabályhely hivatkozás található (az összes dokumentumban).")
    if not jogszabaly_series.empty:
        jogszabaly_counts = jogszabaly_series.value_counts()
        print(f"\nEgyedi jogszabályhelyek száma: {len(jogszabaly_counts)}")
        top_n = 30
        print(f"\nLeggyakrabban hivatkozott {top_n} jogszabályhely:")
        display(jogszabaly_counts.head(top_n))
        plt.figure(figsize=(12, 8))
        jogszabaly_counts.head(top_n).plot(kind='barh')
        plt.title(f'Leggyakrabban hivatkozott {top_n} jogszabályhely')
        plt.xlabel('Hivatkozások száma')
        plt.ylabel('Jogszabályhely')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
    else:
        print("Nem található érvényes jogszabályhely hivatkozás az elemzéshez.")